In [1]:
import os
import numpy as np

import tensorflow as tf

In [75]:
# create a list to all tensorboard log file
# logs_directory = "../../tmp/tensorboard"
logs_directory = "../tensorboard/standalone_tensorboard_10_supervised/"

all_log_dir = os.listdir(logs_directory)

all_log_path = []
for log_dir in all_log_dir:
    log_dir_path = os.path.join(logs_directory, log_dir)
    all_log_path.append(os.path.join(log_dir_path, os.listdir(log_dir_path)[0]))

In [77]:
tags = ["train/acc", "val/acc"]

def get_log_data(path):
    logs = dict(zip(tags, [[] for _ in range(len(tags))]))

    for e in tf.train.summary_iterator(path):
        for value in e.summary.value:
            if value.HasField("simple_value"):
                if value.tag in tags:
                    logs[value.tag].append(value.simple_value)
    
    return logs

# Get all the logs

In [78]:
all_logs = dict(zip(all_log_path, [None for _ in range(len(all_log_path))]))

# for path in all_log_path:
for path in all_log_path:
    all_logs[path] = get_log_data(path)


# Calculate the stats for all the logs

In [79]:
all_logs_stats = dict(zip(all_log_path, [{} for _ in range(len(all_log_path))]))

for path in all_log_path:
    for tag in all_logs[path]:
        try: 
            all_logs_stats[path][tag] = {"max": None, "min": None, "mean": None, "std": None}
            all_logs_stats[path][tag]["max"] = np.asarray(all_logs[path][tag]).max()
            all_logs_stats[path][tag]["min"] = np.asarray(all_logs[path][tag]).min()
            all_logs_stats[path][tag]["mean"] = np.asarray(all_logs[path][tag]).mean()
            all_logs_stats[path][tag]["std"] = np.asarray(all_logs[path][tag]).std()
        except ValueError as e:
            print("Error for tag %s in path %s" % (tag, path))
            print("continue...")
            pass

In [80]:
# search in the stat
def get_of(tag, key, search: str = None) -> list:
    output = []
    
    for path in all_logs_stats.keys():
        if search is not None:
            if search in path:
                output.append(all_logs_stats[path][tag][key])
        else:
            output.append((path, all_logs_stats[path][tag][key]))
    
    return output

In [83]:
target = "max"
response = get_of("val/acc", target)

# trim path from everythinfg irelevant
# response = [(response[i][0].split("/")[4].split("_")[2], response[i][1]) for i in range(len(response))]

# order by run
# response = sorted(response, key=lambda x: x[0].split("/")[4].split("_")[2])

values = np.array([v[1] for v in response])
path = [p[0] for p in response]

values = np.array([i for i in values if i is not None])
print("Overall statistics --------")
print("mean: ", values.mean())
print("std:  ", values.std())
print("min:  ", values.min())
print("max:  ", values.max())
print("")
print("%s values --------" % target)
for p, v in response:
    print(v)

Overall statistics --------
mean:  0.4946961611509323
std:   0.04288830309045337
min:   0.42551183700561523
max:   0.5684915781021118

max values --------
0.5684915781021118
0.500372052192688
0.5055803656578064
0.4393814206123352
0.5496571063995361
0.42551183700561523
0.46339288353919983
0.48882147669792175
0.4830966293811798
0.522656261920929
